In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [2]:
from make_it_talk.models import *
from make_it_talk.scripts import *
from make_it_talk.utils import *

In [6]:
from make_it_talk.data.dataloader import AudioLandmarkDataset
import os

path_to_working_dir = '/Users/antonbeletsky/Downloads/Telegram Desktop'
dts = AudioLandmarkDataset(
    data_dir=path_to_working_dir,
    audio_dir='audio',
    video_dir='video',
    window_size=256
)

Num paths:  0
0 0 0


In [10]:
import torch
dataloader = torch.utils.data.DataLoader(dts_big, batch_size=8, shuffle=True)

In [12]:
batch = None
for b in dataloader:
    batch = b
    break

In [14]:
batch['video'].shape

torch.Size([8, 6012, 256, 256, 3])

In [19]:
pipeline = make_talking_head_pipeline_with_params(root_dir=os.path.join(os.getcwd(), '..'))

In [ ]:
pipeline(batch['audio_content'], batch['audio_speaker'], batch['start_image'])

started audio_to_embedding
Loaded the voice encoder model on cpu in 0.01 seconds.
[[-0.00338745 -0.00140381]
 [-0.00396729 -0.00180054]
 [-0.00338745 -0.00140381]
 ...
 [-0.00299072 -0.00119019]
 [-0.00100708 -0.00039673]
 [ 0.00057983  0.00018311]]


In [6]:
pipeline = make_talking_head_pipeline(
    audio_to_embedding = AudioToEmbedding(os.path.join(os.getcwd(), '..')),
    lstm_speech_content = LSTMSpeechContent(),
    lstm_speaker_aware = LSTMSpeakerAware(),
    mlp_speaker_embedding = MLPSpeakerEmbedding(),
    self_attention_encoder = SelfAttentionEncoder(),
    facial_landmarks_extractor = FacialLandmarksExtractor(),
    mlp_speaker_aware = MLPSpeaker(),
    mlp_speech_content = MLPContent(),
    discriminator = DiscriminatorPlug(68*3, 256, 256)
)

In [7]:
pipeline.content_landmarks_predictor

ContentLandmarkDeltasPredictor(
  (lstm_speech_content): LSTMSpeechContent(
    (lstm): LSTM(80, 256, batch_first=True, dropout=0.5)
  )
  (mlp_speech_content): MLPContent(
    (mlp): Sequential(
      (0): Transpose()
      (1): BatchNorm1d(460, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Transpose()
      (3): Linear(in_features=460, out_features=256, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.5, inplace=False)
      (6): Transpose()
      (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Transpose()
      (9): Linear(in_features=256, out_features=204, bias=True)
      (10): ReLU()
      (11): Dropout(p=0.5, inplace=False)
    )
  )
)

In [8]:
optimizer = torch.optim.Adam(params=pipeline.content_landmarks_predictor.parameters(), lr=1e-3)
loss_fn = LossForContentPredictedLandmarks(lambda_classes=1.0)
metric_fns = []

log = train_content_landmarks_predictor(
        talking_head_pipeline=pipeline,
        optimizer=optimizer,
        dataloader=dataloader,
        n_epochs=2,
        device='cpu',
        loss_function=loss_fn,
        metrics_list=metric_fns,
        training_log=None,
)

Process Video /Users/antonbeletsky/Downloads/Telegram Desktop/obama/video/00000.mp4, len: 6012, FPS: 29.97, W X H: 1920 x 1080


ValueError: axes don't match array

In [22]:
for batch in dataloader:
    print('batch loaded, check audio_content shape:', batch['audio_content'].shape)
    content, speaker = pipeline.audio_to_embedding((batch['audio_content'], batch['audio_speaker']))
    break

/Users/antonbeletsky/opt/anaconda3/lib/python3.9/site-packages/resemblyzer/audio.py:33: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, source_sr, sampling_rate)


Process Video /Users/antonbeletsky/Downloads/Telegram Desktop/obama/video/00000.mp4, len: 6012, FPS: 29.97, W X H: 1920 x 1080
batch loaded, check audio_content shape: torch.Size([1, 8846336, 2])
Loaded the voice encoder model on cpu in 0.02 seconds.


/Users/antonbeletsky/opt/anaconda3/lib/python3.9/site-packages/resemblyzer/audio.py:47: FutureWarning: Pass y=[[-0.00338745 -0.00140381  0.         ...  0.          0.
   0.        ]
 [-0.00396729 -0.00180054  0.         ...  0.          0.
   0.        ]
 [-0.00338745 -0.00140381  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(
/Users/antonbeletsky/opt/anaconda3/lib/python3.9/site-packages/resemblyzer/voice_encoder.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to d

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [9]:
for batch in dataloader:
    preds = pipeline(
        audios_content=batch['audio_content'],
        audios_speaker=batch['audio_speaker'],
        pictures=batch['']
    )
    print(preds[0].shape)
    break

/Users/antonbeletsky/opt/anaconda3/lib/python3.9/site-packages/resemblyzer/audio.py:33: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, source_sr, sampling_rate)


Process Video /Users/antonbeletsky/Downloads/Telegram Desktop/obama/video/00000.mp4, len: 6012, FPS: 29.97, W X H: 1920 x 1080
Loaded the voice encoder model on cpu in 0.02 seconds.


/Users/antonbeletsky/opt/anaconda3/lib/python3.9/site-packages/resemblyzer/audio.py:47: FutureWarning: Pass y=[[-0.00338745 -0.00140381  0.         ...  0.          0.
   0.        ]
 [-0.00396729 -0.00180054  0.         ...  0.          0.
   0.        ]
 [-0.00338745 -0.00140381  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(
/Users/antonbeletsky/Documents/1A/sirius/make-it-talk-project/MakeItScream/MakeItTalkReplication/make_it_talk/models/audio_to_embedding.py:43: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  ains = [item for item in ains if item is not 'tmp.wav']
/Users/antonbeletsky/Documents/1A/si

KeyboardInterrupt: 